In [1]:
from scripts import mapcalc, get_user_POIs

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geocoder

In [3]:
%matplotlib inline

In [4]:
arrests = pd.read_csv('clean_data/arrests_GIS.csv')
schools = pd.read_csv('clean_data/school_list_GIS.csv')
restaurants = pd.read_csv('clean_data/restaurant_list_GIS.csv')
groceries = pd.read_csv('clean_data/grocerystore_list_GIS.csv')
vacancies = pd.read_csv('clean_data/vacancies_GIS.csv')

#Add my own point of interest (JHU Homewood campus, where I work...)
JHU = get_user_POIs.add_user_POI('3400 N Charles St, Baltimore, MD')
Fells = get_user_POIs.add_user_POI('Fells Point, Baltimore, MD')

In [5]:
# compute histograms of arrests and vacant buildings by location
arrest_heat = mapcalc.hist2d_bmoredata(arrests, 0, 0)
vacancy_heat = mapcalc.hist2d_bmoredata(vacancies, 0, 0)

# compute 2D grids of distances to nearest point of interest
restaurant_heat = mapcalc.compute_distances_to_POIs(restaurants)
grocery_heat = mapcalc.compute_distances_to_POIs(groceries[groceries['type'] == 'Full Supermarket'])

# compute 2D grid of distance to Johns Hopkins Homewood campus
# JHU_heat = mapcalc.compute_distances_to_POIs(JHU)

In [5]:
#lowest_val = np.amin(1.0/optimal_locations)
#opt_lat, opt_lon = np.argwhere(1.0/optimal_locations == lowest_val)[0]
#print 'The latitude and longitude coordinates of the best location are:\n%f, %f' % (mapcalc.y[opt_lat], mapcalc.x[opt_lon])

In [6]:
# Wow! That's really close to my house!

In [6]:
from scipy.stats import gaussian_kde

In [190]:
# Boundary conditions for all maps (longitudes as x vals, latitudes as y vals)
lonmin = -76.72
lonmax = -76.52
latmin = 39.19
latmax = 39.38

# number of points along each map edge
# (total number of points is npts**2)
npts = 30

x = np.linspace(lonmin, lonmax, npts)
y = np.linspace(latmin, latmax, npts)

X, Y = np.meshgrid(x, y)
positions = np.vstack([X.ravel(), Y.ravel()])

def kde_map(lon_vec, lat_vec, POIlons, POIlats):
    """
    Gaussian KDE for vectors of points.
    """
    X, Y = np.meshgrid(lon_vec, lat_vec)
    gridpoints = np.vstack([X.ravel(), Y.ravel()])
    
    kernel = gaussian_kde((POIlons, POIlats))

    Z = np.reshape(kernel(gridpoints), X.shape)

    # correct orientation of kernel density estimate
    Z = np.rot90(Z.T)

    return Z / max(Z.flatten())

def gauss_2D(lon_vec, lat_vec, POI_lon, POI_lat, bandwidth):
    """
    Gaussian function for a single point.
    """
    X, Y = np.meshgrid(lon_vec, lat_vec)
    
    # Bandwidth needs to be related to map size!
    # Or maybe the bandwidth of the other maps...
    
    gaussian = np.exp(-((X.ravel()-POI_lon)**2 + 
                        (Y.ravel()-POI_lat)**2)/(2*bandwidth**2))
    
    print max(gaussian)
    Z = np.reshape(gaussian, X.shape)
    Z = np.rot90(Z.T)
    
    return Z

def plot_KDE(lons, lats, kernel_density):
    plt.figure(figsize=(6, 5))
    plt.imshow(
        kernel_density,
        cmap=plt.cm.gist_earth_r,
        extent=[min(lons), max(lons), min(lats), max(lats)]
              )
    plt.axis([min(lons), max(lons), min(lats), max(lats)])
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    cbar = plt.colorbar()
    cbar.ax.set_ylabel('Kernel Density (arb.)')

In [191]:
Z_arrests = kde_map(x, y, arrests['Longitude'], arrests['Latitude'])
Z_schools = kde_map(x, y, schools['Longitude'], schools['Latitude'])
Z_groceries = kde_map(x, y, groceries['Longitude'], groceries['Latitude'])
Z_locs = gauss_2D(x, y, JHU[0][1], JHU[0][0], 1)

0.999997701846


In [209]:
my_map = Z_locs + Z_schools + Z_groceries - Z_arrests
for i in [Z_arrests,Z_schools,Z_groceries,Z_locs,my_map]:
    plot_KDE(x, y, i)

In [207]:
%matplotlib

Using matplotlib backend: MacOSX


In [110]:
my_map[0][0]

0.0